## Boilerplate e API

In [1]:
import json
import io
import base64
import html
import numpy as np
import pandas as pd
import requests
from urllib.request import urlopen
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties

from highlight_text import fig_text
from mplsoccer import Bumpy, FontManager, add_image

In [4]:
df_total_jogos = pd.DataFrame(total_jogos)


## Criacao da classificação geral

In [13]:
def calcular_estatisticas(df):
    estatisticas = {}
    
    for _, row in df.iterrows():
        mandante = row['Mandante']
        visitante = row['Visitante']
        placar_mandante = row['Placar Mandante']
        placar_visitante = row['Placar Visitante']
        
        if mandante not in estatisticas:
            estatisticas[mandante] = {'P': 0, 'V': 0, 'E': 0, 'D': 0, 'Gols': '0:0', 'J': 0}
        if visitante not in estatisticas:
            estatisticas[visitante] = {'P': 0, 'V': 0, 'E': 0, 'D': 0, 'Gols': '0:0', 'J': 0}
        
        mandante_gols = estatisticas[mandante]['Gols']
        visitante_gols = estatisticas[visitante]['Gols']
        mandante_realizados, mandante_sofridos = map(int, mandante_gols.split(':'))
        visitante_realizados, visitante_sofridos = map(int, visitante_gols.split(':'))
        
        mandante_realizados += placar_mandante
        mandante_sofridos += placar_visitante
        visitante_realizados += placar_visitante
        visitante_sofridos += placar_mandante
        
        estatisticas[mandante]['Gols'] = f"{mandante_realizados}:{mandante_sofridos}"
        estatisticas[visitante]['Gols'] = f"{visitante_realizados}:{visitante_sofridos}"
        
        if placar_mandante > placar_visitante:
            estatisticas[mandante]['P'] += 3
            estatisticas[mandante]['V'] += 1
            estatisticas[visitante]['D'] += 1
        elif placar_mandante < placar_visitante:
            estatisticas[visitante]['P'] += 3
            estatisticas[visitante]['V'] += 1
            estatisticas[mandante]['D'] += 1
        else:
            estatisticas[mandante]['P'] += 1
            estatisticas[visitante]['P'] += 1
            estatisticas[mandante]['E'] += 1
            estatisticas[visitante]['E'] += 1
        
        estatisticas[mandante]['J'] += 1
        estatisticas[visitante]['J'] += 1
    
    return estatisticas

estatisticas_times = calcular_estatisticas(df_total_jogos)

df_estatisticas = pd.DataFrame.from_dict(estatisticas_times, orient='index').reset_index()
df_estatisticas.rename(columns={'index': 'Time'}, inplace=True)
df_estatisticas = df_estatisticas[['Time', 'P', 'V', 'E', 'D', 'Gols', 'J']]
df_estatisticas.sort_values(by='P', ascending=False, inplace=True)

df_estatisticas['#'] = range(1, len(df_estatisticas) + 1)
df_estatisticas = df_estatisticas[['#', 'Time',  'P','J', 'V', 'E', 'D', 'Gols']]

In [14]:
df_estatisticas = df_estatisticas.sort_values(by='P', ascending=False)

In [15]:
cols = ['#'] + [col for col in df_estatisticas.columns if col != '#']
df_estatisticas = df_estatisticas[cols]